## Zadanie 1.


#### Klasa do animacji

In [0]:
import numpy as np
import matplotlib.pylab as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

class Animation:

    def __init__(self, name_, x_, y_snapshots_, epochs_, anim_steps_, xlim, ylim):
        self.x = x_
        self.y_snapshots = y_snapshots_
        self.epochs = epochs_
        self.anim_step = anim_steps_
        self.fig = plt.figure()
        self.axes = plt.axes(xlim=xlim, ylim=ylim)
        self.scatter = self.axes.scatter([], [])
        self.text = self.axes.text(2.5, 1.15, '', fontsize=15)
        self.name = name_

    def update(self, i_):
        """Update plot values"""
        self.scatter.set_offsets(np.concatenate((self.x, self.y_snapshots[i_]),
                                                axis=1))
        self.text.set_text(f'Step {i_ * self.anim_step}')
        return self.scatter, self.text,

    def draw(self):
        """Create animation"""
        anim = FuncAnimation(self.fig, self.update,
                             frames=(self.epochs // self.anim_step),
                             interval=40, blit=True)
        anim.save(f'{self.name}.gif', writer='imagemagick')


#### sin(x)
Zadanie próbowałem dla różnych warstw sieci, jednak 1-5-1 daje
wystarczające wyniki. tanh działa znacznie lepiej niż sigmoid
lub relu. Błąd w danym kroku wyświetlany jest w konsoli.

In [0]:
from keras.models import Sequential
from keras.layers import Dense
import matplotlib.pylab as plt
import numpy as np
    
ACTIVATION = 'tanh'

# Dane z wykładu, ale przez małą dziedzinę nie wiele widać
# x = np.linspace(0, 2, 21)
# y = np.sin(x)
# xp = np.linspace(0, 2, 161)

# Ciekawsze dane
x = np.arange(0, np.pi * 2, 0.1)
y = np.sin(x)
xp = x

model = Sequential()
model.add(Dense(1, input_shape=(1,), activation=ACTIVATION))
model.add(Dense(5, activation=ACTIVATION))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='sgd', metrics=['mean_squared_error'])

snapshot = []
epochs = 5000 # tu powinno być więcej ale Google Colab jest wolny
step = 500
for i in range(0, epochs, step):
    model.fit(x, y, epochs=step, batch_size=16, verbose=0)
    yp = model.predict(xp)
    snapshot.append(yp)
    ev = model.evaluate(x, y, verbose=0)
    print(f'step {i + step} error: {ev[1] * 100:.2f}%')
# nie działa w Google Colab, zamiast tego wykres

# anim = Animation(f"sin_1_5_1_{ACTIVATION}", np.reshape(xp, (-1, 1)), snapshot,
                    #  epochs, step, xlim=(0, np.pi*2), ylim=(-1.2, 1.2))
# anim.draw()
plt.plot(xp, model.predict(xp))
plt.show()

#### x^2
Sieć lepiej aproksymowała funkcję gdy nie miała na wejściu 1 neurona.
Sieć działa dobrze jedynie dla ReLU.
Po dodaniu jeszcze jednej warstwy błąd zmniejszył się o połowę.

In [0]:
from keras.models import Sequential
from keras.layers import Dense
import matplotlib.pylab as plt
import numpy as np
from keras.optimizers import Adam

ACTIVATION = 'relu'

x = np.linspace(-50, 50, 26)
y = x ** 2
plt.scatter(x, y)
xp = np.linspace(-50, 50, 101)

model = Sequential()
model.add(Dense(10, input_shape=(1,), activation=ACTIVATION))
model.add(Dense(10, activation=ACTIVATION))
model.add(Dense(10, activation=ACTIVATION))
model.add(Dense(1))
model.compile(optimizer=Adam(), loss='mse')

snapshot = []
epochs = 10000
step = 500
for i in range(0, epochs, step):
   model.fit(x, y, epochs=step, batch_size=100, verbose=0)
   yp = model.predict(xp)
   snapshot.append(yp)
   ev = model.evaluate(x, y, verbose=0)
   print(f'step {i + step} error: {ev * 100:.2f}%')
print('Animacja...')
# anim = Animation(f"x2_1_5_1_{ACTIVATION}", np.reshape(xp, (-1, 1)), snapshot,
#                  epochs, step, xlim=(-50, 50), ylim=(-1, 2500))
# anim.draw()
plt.plot(xp, model.predict(xp))
plt.show()

## Zadanie 2.

In [0]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from keras_retinanet import models
from keras_retinanet.utils.colors import label_color
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption

MODEL_PATH = './resnet50_coco_best_v2.1.0.h5'
IMG_PATH = './atp_pazdziernik2019_035.jpg'


class CameraImageGetter:
    def __init__(self, ffname, cid=0):
        self.camera_id = cid
        self.fallback_filename = ffname

    def get_image(self):
        camera = cv2.VideoCapture(self.camera_id)
        snapshot_ok = False
        image = None
        try:
            for i in range(15):
                snapshot_ok, image = camera.read()
            if not snapshot_ok:
                print("Ups: could not access camera")
                if self.fallback_filename:
                    image = read_image_bgr(self.fallback_filename)
        finally:
            camera.release()
        return image


class FileImageGetter:
    def __init__(self, path):
        self.path = path

    def get_image(self):
        return read_image_bgr(self.path)


class KerasRetina:
    def __init__(self, model_path, image_getter):
        self.labels = {
            0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane',
            5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light',
            10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench',
            14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow',
            20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack',
            25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee',
            30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite',
            34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard',
            37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass',
            41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl',
            46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli',
            51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake',
            56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed',
            60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse',
            65: 'remote', 66: 'keyboard', 67: 'cell phone', 68: 'microwave',
            69: 'oven', 70: 'toaster', 71: 'sink', 72: 'refrigerator', 73: 'book',
            74: 'clock', 75: 'vase', 76: 'scissors', 77: 'teddy bear',
            78: 'hair drier', 79: 'toothbrush'}
        self.model = models.load_model(model_path)
        self.image_getter = image_getter

    def analyze_image(self, img_out):
        # load and prepare image
        image = self.image_getter.get_image()
        image_copy = image.copy()
        image_copy = cv2.cvtColor(image_copy, cv2.COLOR_BGR2RGB)
        image = preprocess_image(image)
        image, scale = resize_image(image)

        # process image
        boxes, scores, labels = self.model.predict_on_batch(np.expand_dims(image, axis=0))
        boxes /= scale

        # draw on image
        for box, score, label in zip(boxes[0], scores[0], labels[0]):
            if score < 0.45:
                break
            color = label_color(label)
            b = box.astype(int)
            draw_box(image_copy, b, color=color)
            caption = "{} {:.3f}".format(self.labels[label], score)
            draw_caption(image_copy, b, caption)
        plt.imsave(img_out, image_copy)


if __name__ == '__main__':
    ig = FileImageGetter(IMG_PATH)
    # ig = CameraImageGetter(IMG_PATH)
    kr = KerasRetina(MODEL_PATH, ig)
    kr.analyze_image(f'img.jpg')


## Zadanie 3.

In [0]:
from keras import layers as lr, Model


def _standard_conv_layer(t, channels, strides, kernel=(3, 3)):
    t = lr.Conv2D(channels, kernel_size=kernel, strides=strides,
                  use_bias=False, padding='same')(t)
    t = lr.BatchNormalization()(t)
    t = lr.ReLU()(t)
    return t


def _pointwise_conv_layer(t, channels):
    return _standard_conv_layer(t, channels, (1, 1), kernel=(1, 1))


def _depthwise_conv_layer(t, channels, strides):
    t = lr.DepthwiseConv2D(kernel_size=(3, 3), strides=strides, use_bias=False,
                           padding='same')(t)
    t = lr.BatchNormalization()(t)
    t = lr.ReLU()(t)
    t = _pointwise_conv_layer(t, channels)
    return t


def _top_layer(t):
    t = lr.GlobalAvgPool2D()(t)
    t = lr.Dense(1000, activation='softmax')(t)
    return t


def MobileNetCifar10():
    _input = lr.Input(shape=(32, 32, 3))
    t = _standard_conv_layer(_input, 32, (2, 2))
    t = _depthwise_conv_layer(t, 64, (1, 1))
    t = _depthwise_conv_layer(t, 128, (2, 2))
    t = _depthwise_conv_layer(t, 128, (1, 1))
    t = _depthwise_conv_layer(t, 256, (2, 2))
    t = _depthwise_conv_layer(t, 256, (1, 1))
    t = _depthwise_conv_layer(t, 512, (2, 2))
    t = _depthwise_conv_layer(t, 512, (1, 1))
    t = _depthwise_conv_layer(t, 512, (1, 1))
    t = _depthwise_conv_layer(t, 512, (1, 1))
    t = _depthwise_conv_layer(t, 512, (1, 1))
    t = _depthwise_conv_layer(t, 512, (1, 1))
    t = _depthwise_conv_layer(t, 1024, (2, 2))
    t = _depthwise_conv_layer(t, 1024, (1, 1))
    t = _top_layer(t)
    return Model(inputs=_input, outputs=t)


In [0]:
from keras.datasets import cifar10
from keras.optimizers import RMSprop
from keras.utils import to_categorical

EPOCHS = 100
BATCH_SIZE = 32
CLASSES = 1000

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
y_train = to_categorical(y_train, CLASSES)
y_test = to_categorical(y_test, CLASSES)

model = MobileNetCifar10()
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(learning_rate=0.0001, decay=1e-6),
              metrics=['accuracy'])
print(model.summary())

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

model.fit(
    x_train, y_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(x_test, y_test),
    shuffle=True
)
print(model.evaluate(x_test, y_test, verbose=1))